## Sagemaker Mxnet/gluon Example
Let's use the final project dataset

In [2]:
import os
import boto3
import sagemaker
from sagemaker.mxnet import MXNet
from mxnet import gluon
from sagemaker import get_execution_role

sagemaker_session = sagemaker.Session()

role = get_execution_role()

#### Reading the dataset from our s3 bucket

In [3]:
import pandas as pd
import pickle

df = pd.read_pickle("./training_data.pkl")
tf = pd.read_pickle("./test_data.pkl")

print(df.head())

     ID                                               data  label
0  1588  [[[255, 255, 255], [255, 255, 255], [255, 255,...      4
1  2011  [[[255, 255, 255], [255, 255, 255], [255, 255,...      1
2  1321  [[[255, 255, 255], [255, 255, 255], [255, 255,...      4
3   669  [[[255, 255, 255], [255, 255, 255], [255, 255,...      3
4  1674  [[[255, 255, 255], [255, 255, 255], [255, 255,...      1


#### Preparing the data for xnet/gluon format

In [4]:
import numpy as np
from skimage.transform import resize

image_vals = df["data"].values
label_vals = df["label"].values

del df

#use tf
del tf

# Create image and label arrays
images = np.zeros((image_vals.shape[0], 3, 224, 224))
labels = np.zeros((label_vals.shape[0]))

# Iterate through the dataframe row by row
for i, (im, label) in enumerate(zip(image_vals, label_vals)):
    # Get image from the data column of the current row
    
    # We need a fixed size input, our images have different sizes, let's pick 224x224.
    # Resize image below
    im = resize(im, output_shape=(224, 224))
    
    # Gluon/mxnet expects images in this format (channel, row, column)
    # This is the opposite of (row, column, channel), let's fix it
    im = np.moveaxis(im, -1, 0)
    
    # Assign the value in the image array
    images[i] = im
    
    # Assign the label in the label array
    labels[i] = label

/home/ec2-user/anaconda3/envs/amazonei_mxnet_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


#### Splitting the data into Training, Validation, Test and saving into s3 bucket

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, labels, stratify=labels, test_size=0.20)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, stratify=y_test, test_size=0.5)

# Save training and validation data locally
np.save('train_data', X_train)
np.save('train_label', y_train)
np.save('validation_data', X_val)
np.save('validation_label', y_val)
np.save('test_data', X_test)
np.save('test_label', y_test)

prefix = 'mla-cv-sagemaker-demo'
bucket = 'gmenende-mla-training'

boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'training/train_data.npy')).upload_file('train_data.npy')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'training/train_label.npy')).upload_file('train_label.npy')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation_data.npy')).upload_file('validation_data.npy')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation_label.npy')).upload_file('validation_label.npy')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test_data.npy')).upload_file('test_data.npy')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test_label.npy')).upload_file('test_label.npy')

In [6]:
# Let's clear the files we saved locally (they are already uploaded to s3 above)
! rm train_data.npy train_label.npy validation_data.npy validation_label.npy test_data.npy test_label.npy

In [7]:
data_location = 's3://{}/{}'.format(bucket, prefix)

In [8]:
m = MXNet("final_project.py",
          role=role,
          train_instance_count=1,
          train_instance_type="ml.p3.2xlarge",
          framework_version="1.2.1",
          py_version = 'py3',
          hyperparameters={'batch_size': 50,
                           'epochs': 50,
                           'learning_rate': 0.02,
                           'patience': 5
                          })

In [9]:
m.fit(data_location)

2019-11-26 16:20:37 Starting - Starting the training job...
2019-11-26 16:20:38 Starting - Launching requested ML instances...
2019-11-26 16:21:37 Starting - Preparing the instances for training............
2019-11-26 16:23:13 Downloading - Downloading input data......
2019-11-26 16:24:38 Training - Training image download completed. Training in progress..2019-11-26 16:24:38,906 INFO - root - running container entrypoint
2019-11-26 16:24:38,906 INFO - root - starting train task
2019-11-26 16:24:38,926 INFO - container_support.training - Training starting
2019-11-26 16:24:40,123 WARNING - mxnet_container.train - #033This required structure for training scripts will be deprecated with the next major release of MXNet images. The train() function will no longer be required; instead the training script must be able to be run as a standalone script. For more information, see https://github.com/aws/sagemaker-python-sdk/tree/master/src/sagemaker/mxnet#updating-your-mxnet-training-script.#033
2

In [10]:
predictor = m.deploy(instance_type='ml.m4.xlarge', initial_instance_count=1, endpoint_name='mla-cv-endpoint')

--------------------------------------------------------------------------------------!

## Let's test this endpoint
We will read the test data from our S3 buckets and call the endpoint we just created.

In [12]:
import numpy as np
from io import BytesIO
import mxnet.ndarray as nd
import boto3

sm = boto3.client('s3')
obj = sm.get_object(Bucket='gmenende-mla-training', Key='mla-cv-sagemaker-demo/test/test_data.npy')
X_test = np.load(BytesIO(obj['Body'].read()))

obj = sm.get_object(Bucket='gmenende-mla-training', Key='mla-cv-sagemaker-demo/test/test_label.npy')
y_test = np.load(BytesIO(obj['Body'].read()))

payload = X_test[:4]
payload = bytearray(payload)

runtime = boto3.Session().client(service_name='runtime.sagemaker')
response = runtime.invoke_endpoint(EndpointName="mla-cv-endpoint", 
                                   ContentType='application/x-image', 
                                   Body=payload)

In [13]:
print(response['Body'].read())

b'[2.0, 2.0, 4.0, 4.0]'
